In [1]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse


os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")
from logger import logger 
from config import cfg


cfg.N_CLUSTER_RATION = 200

cfg.EDIT_DISTANCE_THRESH_LONG = 0.3
cfg.EDIT_DISTANCE_THRESH_SHORT = 0.2
cfg.LONG_URL_THRESH = 100
cfg.SHORT_URL_THRESH = 20

cfg.MIN_SUBSTRING_SIZE = 12
cfg.SINGLE_REGEX_SIZE = 20
cfg.TOTAL_REGEX_SIZE = 20
cfg.SINGLE_REGEX_SIZE_RATIO = 0.2
cfg.TOTAL_REGEX_SIZE_RATIO = 0.2

from file_io import load_urls, dump_urls
from preprocess import make_ip_url_map
from vectorize import make_vectorize
from clustering import make_ip_cluster, make_string_distance_cluster
from regex_url import url_regex_extract, url_regex_check, url_regex_publish

import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import file_io

In [2]:
urls = load_urls("../../data/VirusTotalnew/malicious/total3-ma.txt", csv = False)
urls = list(set(urls))
print len(urls)

2018-04-19 11:00:59 file_io.py [line:52] DEBUG	URLs Count:	237128


43789


In [3]:
file_io.dump_urls(urls, "../../data/EXP_VirusTotal7/malware.csv")

2018-04-19 11:01:01 file_io.py [line:24] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/malware.csv
2018-04-19 11:01:01 file_io.py [line:33] DEBUG	URLs has been dump	../../data/EXP_VirusTotal7/malware.csv


In [4]:
url_domain, url_path, url_param = preprocess.make_url_classification(urls)
print len(url_domain)
print len(url_path)
print len(url_param)

2018-04-19 11:01:03 preprocess.py [line:75] DEBUG	1 urls length do not meet size req 20


0
43124
664


In [5]:
df_vector =  make_vectorize(url_path, domain=True, path=True, param=False, dump = False)

2018-04-19 11:01:11 vectorize.py [line:77] DEBUG	vectorization complete! data shape:	(43124, 256)


In [6]:
cc_dict = clustering.make_hier_cluster(df_vector, dump = False)

2018-04-19 11:01:11 clustering.py [line:173] DEBUG	begin to make hierarchical cluster, data_size: 43124 n_clusters: 21
2018-04-19 11:06:28 clustering.py [line:190] DEBUG	hcluster done!


In [7]:
file_io._dump_kmeans_hier_cluster_data(cc_dict, "../../data/EXP_VirusTotal7/cluster_hier.json")

2018-04-19 11:07:49 file_io.py [line:150] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/cluster_hier.json
2018-04-19 11:07:49 file_io.py [line:154] DEBUG	cluster has been dump	../../data/EXP_VirusTotal7/cluster_hier.json


In [8]:
cc_list = file_io._load_kmeans_hier_cluster_data("../../data/EXP_VirusTotal7/cluster_hier.json")

2018-04-19 11:07:52 file_io.py [line:133] DEBUG	K-means/hier data has been loaded	../../data/EXP_VirusTotal7/cluster_hier.json


In [10]:
clustering.make_string_distance_cluster(data = cc_list, 
                                            n_jobs = 8,
                                            metric = "distance",  
                                            file_path = "../../data/EXP_VirusTotal7/cluster_distance.json")

index 0 0/1431
index 0 0/193
index 0 0/473
index 0 0/325
index 0 0/1325
index 0 0/8418
index 0 0/776
index 0 0/29882
index 100 100/193
index 0 0/106
index 100 100/106
index 0 0/99
index 0 0/24
index 0 0/15
index 0 0/15
index 0 0/11
index 0 0/9
index 0 0/9
index 0 0/5
index 0 0/4
index 0 0/2
index 0 0/1
index 0 0/1
index 200 200/325
index 200 200/776
index 200 200/473
index 100 100/1325
index 300 300/1431
index 300 300/776
index 300 300/473
index 500 500/1431
index 600 600/776
index 600 600/1431
index 100 100/8418
index 200 200/1325
index 300 300/1325
index 200 200/8418
index 400 400/1325
index 100 100/29882
index 500 500/1325
index 300 300/8418
index 600 600/1325
index 700 700/1325
index 800 800/1325
index 900 900/1325
index 1100 1100/1325
index 1300 1300/1325
index 200 200/29882
index 500 500/8418
index 900 900/8418
index 400 400/29882
index 1400 1400/8418
index 500 500/29882
index 1800 1800/8418
index 600 600/29882
index 2200 2200/8418
index 700 700/29882
index 3700 3700/8418
index 4

2018-04-19 11:13:30 file_io.py [line:103] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/cluster_distance.json
2018-04-19 11:13:30 clustering.py [line:109] DEBUG	string distance time cost:	330.109862


In [11]:
regex_list = regex_url.url_regex_extract(input_file_path = "../../data/EXP_VirusTotal7/cluster_distance.json",
                                         output_file_path = "../../data/EXP_VirusTotal7/regex_raw.txt")

2018-04-19 11:16:04 file_io.py [line:172] DEBUG	Cluster Data has been loaded	../../data/EXP_VirusTotal7/cluster_distance.json
2018-04-19 11:16:04 regex_url.py [line:210] DEBUG	total cluster num:	6046
2018-04-19 11:16:04 regex_url.py [line:212] DEBUG	big cluster:	1339
2018-04-19 11:16:04 regex_url.py [line:214] DEBUG	small cluster:	1169
2018-04-19 11:16:04 regex_url.py [line:215] DEBUG	single one:	3538
2018-04-19 11:16:04 regex_url.py [line:216] DEBUG	cluster size detail:	Counter({1: 3538, 2: 757, 3: 264, 4: 148, 5: 113, 6: 65, 8: 57, 7: 55, 9: 42, 10: 30, 12: 29, 13: 28, 11: 24, 16: 22, 15: 21, 17: 20, 14: 19, 42: 17, 19: 16, 18: 14, 20: 13, 21: 13, 31: 13, 40: 12, 27: 11, 30: 11, 32: 11, 38: 10, 22: 9, 23: 9, 24: 9, 25: 9, 35: 9, 41: 9, 45: 9, 56: 9, 61: 9, 29: 8, 34: 8, 36: 8, 37: 8, 33: 8, 26: 7, 47: 7, 53: 7, 68: 7, 28: 6, 43: 6, 44: 6, 55: 6, 59: 6, 80: 6, 82: 6, 101: 6, 58: 6, 79: 6, 49: 5, 52: 5, 93: 5, 98: 5, 162: 5, 48: 4, 60: 4, 63: 4, 64: 4, 66: 4, 78: 4, 91: 4, 96: 4, 97: 4

step: 2200
step: 2400
step: 400
step: 2600
step: 2800
step: 3800
step: 4400
step: 4600
step: 4800
step: 5000
step: 5200
step: 5400
step: 5600
step: 5800
step: 6000


2018-04-19 11:17:04 regex_url.py [line:225] DEBUG	extract regex count:	1158
2018-04-19 11:17:04 regex_url.py [line:226] DEBUG	extract regex time cost:	60.070910
2018-04-19 11:17:04 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/regex_raw.txt
2018-04-19 11:17:04 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal7/regex_raw.txt


In [12]:
regex_url.url_regex_check(input_file_path      = "../../data/EXP_VirusTotal7/regex_raw.txt", 
                      test_benign_file_path    = "../../data/sangfor/safe.csv",
                      test_malicious_file_path = "../../data/EXP_VirusTotal7/malware.csv",
                      result_file_path         = "../../data/EXP_VirusTotal7/regex_result.txt", 
                      n_jobs = 8)

2018-04-19 11:17:23 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal7/regex_raw.txt
2018-04-19 11:17:24 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv
2018-04-19 11:17:24 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/EXP_VirusTotal7/malware.csv


batch index 0	sample index 0	FP 0	TP 120	flv\.softpzivrubajjui\.net/n/3\.1\.1
batch index 0	sample index 1	FP 0	TP 2	\.ru/download/3a/W05YVFESKDU2M35qaSEnOykuP2M4PDU6PSk1eyQidz4/LwM3LTAOXhEXBlhSQQwADlZdXF(.)+UBIADSULHQ8KEeXz3erguKDh4eXv1OXpswC
batch index 0	sample index 2	FP 0	TP 1	dl\.downf468\.com/n/3\.1\.13(.)+/11999301/silvertv3\.0\.exe
batch index 0	sample index 3	FP 0	TP 133	hao\.jhczs\.com/141492(.)+%E6%92%AD%E6%94%BE%E5%99%A8_1
batch index 1	sample index 0	FP 0	TP 49	%C3%A6%C2%92%C2%AD%C3%A6%C2%94(.)+%C3%A5%C2%99
batch index 0	sample index 4	FP 0	TP 30	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/222529105\.(.)+/msniEasyPDFCombine/EasyPDFCombine\.
batch index 1	sample index 1	FP 0	TP 3	\.ru/output/4pz82blknp2bz2f5uhm7uem2bv2n72b3812fdl5lfu4ett12bhstq3d3d/02/(.)+/88/torrent/
batch index 0	sample index 5	FP 0	TP 2	\.ru/output/m4ehg8nc3jeenzqdlzzdgybcljahriuen8yvmp2bwrjqxzg3d3d/02/(.)+/f3/archive/
batch index 1	sample index 2	FP 0	TP 197	dl\.downe468\.com/n/3\.0\.26/10
ba

batch index 1	sample index 16	FP 0	TP 52	static\.bicdn\.com/installers/(.)+_downloader_by_network_yesadvertising_1/
batch index 3	sample index 9	FP 0	TP 5	\.ru/output/92bvr76wwspvo8fbx2bfcx7eqw2bprrwofy86d59vp6wpb7og3d3d/02/c
batch index 0	sample index 20	FP 0	TP 2	\.ru/download/a5/xNTEwMDczoHF2tvAi52c0trE1N3KlNbFkNjTr66w7vbx6LW95q2uuJKkvL\+/7aCgt\+vj/r2zv\+Hs7\+f(.)+wY2RnrScjJybnpSArJ2Ry9GekJaeo5SawgC
batch index 5	sample index 2	FP 0	TP 1	dl\.downf468\.com/n/3\.0\.26/4791661/le(.)+monopoly\.exe
batch index 2	sample index 13	FP 0	TP 9	speedapps\.com/builds/
batch index 1	sample index 17	FP 0	TP 1	dl\.downloadahceiduphoth\.com/n(.)+/12852267/Adobe\+Acrobat\+Professional\.exe
batch index 4	sample index 6	FP 0	TP 8	media\.vitkvitk\.com/xmlstatic/installers/software/freesofttoday/setup_vittalia_fst_
batch index 5	sample index 3	FP 0	TP 2	1-monster\.ru/6vb28ritrett8ohn8azg9/uv5ovu56zw963l5/bd\+u/uvetr7ufd6\+a/torrent/140983938/482997696/pinnacle_studio_17_ultimate_tfile_ru\.exe
batch index

batch index 7	sample index 4	FP 0	TP 4	get\.coedbest\.com/n/3\.1\.5(.)+85/SopCast\.exe
batch index 1	sample index 26	FP 0	TP 4	de\.saeas-about-home\.com/80/275/ct2754380/30d597b70671477c9a3c2edff6942a32/(.)+tub1\.3\.9\.0\.140504\.01/14-10-2
batch index 5	sample index 12	FP 0	TP 10	dde\.de\.saeas-about-home\.com/(.)+/downloads/prod/dde1\.4\.0\.(.)+/utorrent\.exe
batch index 0	sample index 30	FP 0	TP 4	flyingsalem\.club/installs/231
batch index 2	sample index 23	FP 0	TP 206	dl\.cdnstatic1985\.com/n/3\.
batch index 6	sample index 8	FP 0	TP 2	dlc\.ecozoro\.ru/download/c3/oramrq6\+rOejuLm\+9f/\+tLymtrOk9quv9rCysr7Ok5fMg4CSuIKahYXTnpqNzcXUl52Ry8bJwMvOycvNyDInZ3twWnZmen1kbn5SZ2stN3R6eHBJfnwkAKzYlA/windows_loader_v2_2_2\.exe
batch index 4	sample index 15	FP 0	TP 5	\.hflddz\.com/anchor/zb_anchor\.exe
batch index 3	sample index 19	FP 0	TP 2	\.ru/output/oiymimh9ftylpds8nd18icd9ntcmdso2fpm00oz43dts2bw3d3d/02/(.)+/52/torrent/(.)+-_torrentino\.exe
batch index 7	sample index 5	FP 0	TP 2	\.ru/output/f

batch index 5	sample index 20	FP 0	TP 2	u\.ru/output/f2njzy04ohngex55cxg5zwi4chjjsg96eyhxfntysh5zkg3d3d/02/(.)+_zaycev_net\.exe
batch index 4	sample index 23	FP 0	TP 11	report\.topsadon\.com/report_exe_agent\.asp
batch index 6	sample index 16	FP 0	TP 2	\.moilru\.ru/output/jdg4phzjyyg7iiuikinipjljkyk4ezqhihmqjsapeyuocq3d3d/0
batch index 3	sample index 27	FP 0	TP 5	\.ru/output/eq0ncunwvh0ofxaxhxzxcwxwhhwnjgeufuyfebucjhadra3d3d/0
batch index 1	sample index 35	FP 0	TP 13	zhainan\.xzzxjly\.com/down/(.)+%C3%83%C2%A6%C3%82%C2%92%C3%82%C2%AD%C3%83%C2%A6%C3%82%C2%94%C3%82(.)+%C3%83%C2%A5%C3%82%C2%99%C3%82
batch index 7	sample index 13	FP 0	TP 23	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/207743773\.LM(.)+/msniFromDocToPDF/FromDocToPDF\.
batch index 5	sample index 21	FP 0	TP 3	dl01\.fabdmr\.com/n/3\.0\.(.)+/minecraft\.exe
batch index 0	sample index 39	FP 0	TP 5	ruinforums\.com/stubs/
batch index 2	sample index 32	FP 0	TP 2	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/2(.)+\.T

...

batch index 3	sample index 111	FP 0	TP 12	\.sltbzy\.com/anchor/zb_anchor\.exe
batch index 6	sample index 101	FP 0	TP 2	dlc\.kozainfo\.ru/download/3f/sy8zmcyqmwsvpd06cwnikcaimjcgejgvmtq2nsluaxatfu4fbha6dbqhb1uyga9tw1yvgxdjredpte9dte9esfnl\+fbc9ot08\+bs\+ntl6bop9vj\+9sv88qoanb99/skachat_konsultant_plyus_federalnaya_sborka_ot_09_05_2014g_2014_rus_torrent_bez_registracii\.exe
batch index 7	sample index 99	FP 0	TP 3	\.ru/output/k42bpi8hu1j2bmlzkvnztviy7unj6ppiowl8sdkpeepjkfxg3d3d/01/
batch index 2	sample index 117	FP 0	TP 2	91\.217\.82\.188/forum/load/1
batch index 1	sample index 121	FP 0	TP 5	\.ru/output/ztls1pyjiclrym2fiwmmi1nojwcps2bd7lypnaz8rd2bc2fcmw3d3d/02/
batch index 0	sample index 125	FP 0	TP 1	get\.unitebest\.com/n/3\.1\.54(.)+/13262614/Minecraft\.exe
batch index 4	sample index 110	FP 0	TP 2	221\.180\.22\.194:[0-9]{2,}/down/
batch index 5	sample index 107	FP 0	TP 3	\.ru/download/bd/ydHNsqSst\+mtsrO48\+XkqqK8rLWi/KWnsL24orj0qanyubqUvoiQi4vZlJyL19/KiYeLzcDD(.)+dmYWKoHBgcHdqYHRYYW03L

batch index 7	sample index 108	FP 0	TP 2	dlc\.kozagroup\.ru/download/4b/OCk5Oz99OSYnJG95eD42KDg5LnAyGUwEDwsKFEpaXUQZGUIJCgQuGAAbG0kEDBtHT1oZFxu9sLO7tbC(.)+3p9frQ4PDg5/rw5Mjx/ae9\+vTy\+v/Ixp4Ak9/F/utorrent\.exe
batch index 3	sample index 120	FP 0	TP 2	k\.sdfsd\.dhcp\.biz:[0-9]{2,}/2\.exe
batch index 6	sample index 110	FP 0	TP 206	sub\.spirlymo\.com/installers/cli/14(.)+ip_downloader-
batch index 0	sample index 134	FP 0	TP 5	dlc\.ramastroy\.ru/download/e3/kIGRk5fFgZ6fnNfBwJaegJCRhtiBi5yRlIac0I11LmVmcFpsdGdnNXh4bzM7NnV7dykkJyAhL(.)+OUVZVnxURFRTRkxYdEVJEwlWWF5Wa1xSCgCQ6VQ/obnovleni\.exe
batch index 5	sample index 115	FP 0	TP 2	e\.info/addons/pvtzd_agent_setup\.exe
batch index 2	sample index 126	FP 0	TP 7	wrapapp\.net/apps/mymusictube/downloads/(.)+/mymusictubesetup
batch index 4	sample index 119	FP 0	TP 554	downloadcdn\.betterinstaller\.com/installers/(.)+_downloader_by_
batch index 1	sample index 130	FP 0	TP 6	downloadcdn\.betterinstaller\.com/installers/(.)+Font_downloader_by_CopyFonts\.

batch index 3	sample index 128	FP 0	TP 2	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/226328439\.(.)+/msniTestForSpeed/TestForSpeed\.
batch index 1	sample index 138	FP 0	TP 3	\.ru/output/ljiynnxpasixkc8oiclondnpismygt4rknkglyojgs8iew3d3d/0(.)+/46/torrent/
batch index 6	sample index 118	FP 0	TP 14	dl\.downloadaesaenineipi\.com/n/3\.2\.2/1
batch index 4	sample index 127	FP 0	TP 348	dl\.media1982cdn\.com/n/3\.1\.
batch index 2	sample index 134	FP 0	TP 2	artmatic\.com\.br/js/Me
batch index 7	sample index 117	FP 0	TP 3	\.ru/download/2f/W19DQFZaQRtfTE1KARMSWFAyIicwaj(.)+piPztgNzQmDD4mOTlnKi45YWl4OwkFX1JV(.)+LCxcELgISBgEYEgomEx9BWxgW7OTd6uC4A
batch index 5	sample index 124	FP 0	TP 2	1-monster\.ru/a3d3czkslgvscwbmcc1hdnouzwpvzi1xdixkzndce25vpgvqb2zcamc\+/torrent/31835395/482993196/nhl_eastside_hockey_manager_2007_skachat_igry_cherez_torrent_-_s\.exe
batch index 0	sample index 143	FP 0	TP 2	121717112406109909115105\.yz88\.org/
batch index 3	sample index 129	FP 0	TP 4	dl01\.faadmr\.com/

batch index 5	sample index 134	FP 0	TP 3	down\.cncrk\.com:[0-9]{2,}/soft/software/
batch index 6	sample index 129	FP 0	TP 2	a\.ru/output/mi4ukmb1dt4tndm0pdv0kc91pt8ubsi3nmu8mzy2fbtm2bzw3d3d/02/
batch index 7	sample index 128	FP 0	TP 716	static\.bicdn\.com/installers/(.)+r_downloader_by_network_
batch index 4	sample index 138	FP 0	TP 2	\.com/n/3\.1\.56/12721228/
batch index 3	sample index 140	FP 0	TP 198	\.ytqihang\.com/soke/m_soke\.exe
batch index 5	sample index 135	FP 0	TP 10	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/23300(.)+\.TTAB02\.1/nsis/77(.)+-TTAB02\.1/17080(.)+finder/online
batch index 6	sample index 130	FP 3	TP 72	l\.gxjsxq\.com/download/
batch index 7	sample index 129	FP 0	TP 169	dl\.cdn1981media\.com/n/3\.
batch index 4	sample index 139	FP 0	TP 2	www\.traducegratis\.com/downloads/installer_(.)+_0_Spanish\.exe
batch index 3	sample index 141	FP 0	TP 4	\.ru/output/8ozs6kk3t2fzv9vh22fve26u232f2f3sx2bd19kf2b8ft9x2fh8pq3d3d/02/
batch index 5	sample index 136	FP 0	TP 2

2018-04-19 11:21:20 file_io.py [line:249] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/regex_result.txt
2018-04-19 11:21:20 file_io.py [line:255] DEBUG	Check Result has been dump	../../data/EXP_VirusTotal7/regex_result.txt


In [2]:
df = regex_url.url_regex_publish(result_file_path = "../../data/EXP_VirusTotal7/regex_result.txt",
                       publish_file_path = "../../data/EXP_VirusTotal7/regex_publish.txt",
                       publish_fp_thresh = 100,
                       publish_tp_thresh = 1,
                       publish_ratio = 1000,
                       publish_ascore = 0,
                       publish_pscore = 1)

df = df.loc[df.fp>0]
df = df.set_index('regex')
df

2018-04-21 09:50:47 file_io.py [line:276] DEBUG	check Data has been loaded	../../data/EXP_VirusTotal7/regex_result.txt
2018-04-21 09:50:47 regex_url.py [line:355] DEBUG	regular expression publish	1156
2018-04-21 09:50:47 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/regex_publish.txt
2018-04-21 09:50:47 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal7/regex_publish.txt


,fp,tp,pscore,ascore,ratio
regex,,,,,
%E6%92%AD%E6%94%BE%E5%99%A8_1,2,338,0.931034,0,0.005947
d1\.wanyxbox\.com/tg/200021/s1g,3,5,0.000000,5,0.600798
%E6%92%AD%E6%94%BE%E5%99%A8_,5,411,0.964286,0,0.012189
l\.gxjsxq\.com/download/,3,72,0.000000,4,0.041800
/Microsoft\+Office\+201,8,65,0.000000,1,0.123212


In [17]:
regex_list_publish = list(df.index)
file_io._dump_regex_list(regex_list_publish,
                 "../../data/EXP_VirusTotal7/regex_publish.txt")

2018-04-19 11:58:51 file_io.py [line:207] DEBUG	OLD DATA FIND! REMOVING	../../data/EXP_VirusTotal7/regex_publish.txt
2018-04-19 11:58:51 file_io.py [line:212] DEBUG	Regex has been dump	../../data/EXP_VirusTotal7/regex_publish.txt


In [18]:
predict_malicious, predict_dict = regex_url.malicious_url_predict(
                      input_file_path = "../../data/sangfor/safe.csv",
                      regex_file_path = "../../data/EXP_VirusTotal7/regex_publish.txt")

2018-04-19 11:58:55 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal7/regex_publish.txt
2018-04-19 11:58:56 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe.csv


batch index 7	sample index 0	hit url 3	d1\.wanyxbox\.com/tg/200021/s1g
batch index 7	sample index 1	hit url 3	l\.gxjsxq\.com/download/


In [29]:
import urllib
print urllib.unquote_plus("%E6%92%AD%E6%94%BE%E5%99%A8_")

播放器_


In [19]:
len(predict_malicious)

6

In [20]:
predict_malicious

['http://dl.gxjsxq.com/download/setup_27@9.exe',
 'http://cl.gxjsxq.com/download/setup_27@9.exe',
 'http://dl.gxjsxq.com/download/%E4%B8%AD%E4%BF%A1%E5%BB%BA%E6%8A%95%E7%BD%91%E4%B8%8A%E4%BA%A4%E6%98%93%E6%9E%81%E9%80%9F%E7%89%88_48@50200.exe',
 'http://d1.wanyxbox.com/tg/200021/s1g151.exe',
 'http://d1.wanyxbox.com/tg/200021/s1g1658.exe',
 'http://d1.wanyxbox.com/tg/200021/s1g20440.exe']

In [23]:
import pprint
for i in predict_dict:
    print i
    pprint.pprint(predict_dict[i])
    print "\n"

%E6%92%AD%E6%94%BE%E5%99%A8
[['http://19449.xc.cangpie.com/down/%E5%85%88%E9%94%8B%E5%BD%B1%E9%9F%B3%E6%92%AD%E6%94%BE%E5%99%A8@133_39526.exe'],
 ['http://14614.xc.chuairan.com/down/%E8%85%BE%E8%AE%AF%E8%A7%86%E9%A2%91%E6%92%AD%E6%94%BE%E5%99%A8@626_4686.exe'],
 ['http://onlinedown.down.123ch.cn/download/SWF%E6%92%AD%E6%94%BE%E5%99%A8_1@95201.exe'],
 ['http://19387.xc.cangpie.com/xiaz/Realcodec%E6%92%AD%E6%94%BE%E5%99%A8%E6%8F%92%E4%BB%B62.37%E5%AE%98%E6%96%B9%E6%9C%80%E6%96%B0%E7%89%88@284_88129.exe'],
 ['http://19291.xc.cangpie.com/xiaz/Realcodec%E6%92%AD%E6%94%BE%E5%99%A8%E6%8F%92%E4%BB%B62.37%E5%AE%98%E6%96%B9%E6%9C%80%E6%96%B0%E7%89%88@284_88129.exe'],
 ['http://19387.xc.gongnou.com/xiaz/Realcodec%E6%92%AD%E6%94%BE%E5%99%A8%E6%8F%92%E4%BB%B62.37%E5%AE%98%E6%96%B9%E6%9C%80%E6%96%B0%E7%89%88@284_88129.exe'],
 ['http://19429.xc.gongnou.com/down/potplayer%E6%92%AD%E6%94%BE%E5%99%A832%E4%BD%8D@133_4017.exe'],
 ['http://18231.url.016272.com/down/%E7%88%B1%E5%A5%87%E8%89%BA%E6%92%AD%E6%9

In [22]:
predict_malicious, predict_dict = regex_url.malicious_url_predict(
                      input_file_path = "../../data/sangfor/safe0416.csv",
                      regex_file_path = "../../data/EXP_VirusTotal7/regex_publish.txt")

2018-04-19 12:01:11 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal7/regex_publish.txt
2018-04-19 12:01:12 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/safe0416.csv


batch index 0	sample index 0	hit url 0	flv\.softpzivrubajjui\.net/n/3\.1\.1
batch index 0	sample index 1	hit url 0	dl\.downf468\.com/n/3\.1\.13(.)+/11999301/silvertv3\.0\.exe
batch index 0	sample index 2	hit url 0	hao\.jhczs\.com/141492(.)+%E6%92%AD%E6%94%BE%E5%99%A8_1
batch index 0	sample index 3	hit url 0	\.ru/output/m4ehg8nc3jeenzqdlzzdgybcljahriuen8yvmp2bwrjqxzg3d3d/02/(.)+/f3/archive/
batch index 1	sample index 0	hit url 0	dl\.wasdmr\.com/n/3\.0\.26/5363646/arma\+2\.exe
batch index 0	sample index 4	hit url 0	91\.217\.82\.188/forum/load/1
batch index 1	sample index 1	hit url 0	dlc\.lola-trade\.ru/download/66/FQIcHBpGBBkaH0peXRUbBxUSC1cXAhoRERDys7at9vCp4O391eH/4uCw4\+XwrqCz8v78pKuqpKioqZ(.)+CwN7T99nL2djDy93v2NaOktPf293m09\+BAJsqaA/obnovleni\.exe
batch index 0	sample index 5	hit url 0	download-instantly\.com/isn/js/http:/software-files-a\.cnet\.com/s/software/13/
batch index 1	sample index 2	hit url 0	down\.shuyeer\.net/dudu/(.)+%C2%BF%C2%BD(.)+%C2%BF%C2%BD(.)+%C2%BF%C2%BD(.)+%C2%BF%

batch index 2	sample index 14	hit url 0	\.ru/output/9enp7aeysvnq82ftz2b2fkz72biy2bvjpwuxw8al79ph4wvt5oa3d3d/0(.)+/9d/torrent/
batch index 3	sample index 10	hit url 0	d\.epubgratis\.me/download/(.)+-george-r-r-martin-epubgratis\.me\.exe
batch index 1	sample index 18	hit url 0	218\.38\.12\.110/tool/dtsys\.exe
batch index 0	sample index 22	hit url 0	dl2\.vid4installer\.com/download/
batch index 4	sample index 6	hit url 0	dl\.cdnmedia1980\.com/n/3\.1\.5
batch index 5	sample index 3	hit url 0	\.ru/download/2a/XkReX0tBRBxaR0BFDBgXX1VJX1hNETI0by8rKSdpOjxlLCk5ESUjPjxsJyE0amx/PjI4YG9uW(.)+dODBIfMx0PHQQfFwErHBJKXh8TFxkiFxu9AK
batch index 3	sample index 11	hit url 0	zn\.tybests\.com/down/zhainan/%(.)+C%D7%A8%D3%C3%B2%A5%B7%C5%C6%F7
batch index 2	sample index 15	hit url 0	foodchinesecucumberboss\.de/(.)+/movie1080p\.mkv\.exe
batch index 1	sample index 19	hit url 0	\.ru/output/92bvr76wwspvo8fbx2bfcx7eqw2bprrwofy86d59vp6wpb7og3d3d/0
batch index 0	sample index 23	hit url 0	dl\.downloadahceiduphoth\.c

batch index 2	sample index 24	hit url 0	dlc\.online-floor\.ru/download/5a/ljqulzsxfewkfxavxehhdwuzdwgdqqiexx8bgrdzcgxvhbkjirxz7uy89/hkuryv7ulosl\+\+qac(.)+g\+/olvw\+3/7dtpx9h7zmkajs/dx8nyx8unak
batch index 4	sample index 16	hit url 0	\.ru/output/62ff387msrofm92blq72bf07a3x9qzk5vfc52bz07e2fs4ufc2b2b7v3lc86ue2b/00/13/a9/83/archive/archive\.exe
batch index 1	sample index 28	hit url 0	ffdy\.51web8\.net/free_383_1
batch index 3	sample index 20	hit url 0	down\.pengan119\.com/tg/v10\.6\.1
batch index 5	sample index 12	hit url 0	dl\.downf468\.com/n/3\.0\.26/114(.)+/d3dcompiler_43\.exe
batch index 6	sample index 9	hit url 0	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/231148733\.TTAB02\.1/nsis/770402-TTAB02\.1/170803120308524/msniFreeDirectionsOnline/FreeDirectionsOnline\.
batch index 0	sample index 32	hit url 0	file\.utilz\.net/dst/TopTool_TT
batch index 7	sample index 5	hit url 0	\.ru/output/ar0dgvngrg0ebwahdwzhgxxgdgwdnheebvypaaumnganva3d3d/0
batch index 2	sample index 25	hit url 0	

batch index 6	sample index 17	hit url 0	tazbox\.zapto\.org/downloader/dump/
batch index 5	sample index 20	hit url 0	\.ru/download/ae/2sDCw9fdwJjew8zJgJST29HNo6Sx7b(.)+vlvrjhqLWljbmnurjoq6245uj7uraE3NPS(.)+vKiJabr4GTgYCbk4WnkJ7G2puXk2Vea2c5A
batch index 4	sample index 25	hit url 0	free\.nice-host\.biz/download/8f49801a-035e/fifa\+13\.exe
batch index 2	sample index 33	hit url 0	dlc\.ramastroy\.ru/download/34/(.)+ZUSUpPem5tJSs3JSI7Zzw4KSYhNTF/ICZ7MjMjBzMpNDJiLSsCXFZFAAwCWllY(.)+UFgwBKQcZCw4VGQ8hFuS8pOXt6ePY4e23AJ
batch index 7	sample index 13	hit url 0	\.com/down/zhainan/%E4%B8%8B%E8%BD%BD%E6%89%93%E5%BC%80%E6%92%AD%E6%94%BE_8_
batch index 1	sample index 37	hit url 0	\.ru/output/v0tltwuqeftiuvzrwvartuoqwfplyedsuwbzvlnayfzbag3d3d/02/6d/1e/3f/setup/updatesetup\.exe
batch index 6	sample index 18	hit url 0	flv\.flvfastdown\.com/n/3\.(.)+/12912908/flashplayer\.exe
batch index 3	sample index 29	hit url 0	dl\.cdn1981media\.com/n/3\.1\.5
batch index 0	sample index 41	hit url 0	dlc\.info-molk\.ru/

...

batch index 4	sample index 103	hit url 0	\.ru/output/hqebbu9awheqarqcgrecg1shafoseaeqerocgxkafbeqdrgzkkzkhbfi/00/
batch index 7	sample index 90	hit url 0	/filestock\.blob\.core\.windows\.net/download/Torn_injection_fs\.exe
batch index 1	sample index 114	hit url 0	artmatic\.com\.br/js/Me
batch index 6	sample index 95	hit url 0	dlc\.kozainfo\.ru/download/3f/sy8zmcyqmwsvpd06cwnikcaimjcgejgvmtq2nsluaxatfu4fbha6dbqhb1uyga9tw1yvgxdjredpte9dte9esfnl\+fbc9ot08\+bs\+ntl6bop9vj\+9sv88qoanb99/skachat_konsultant_plyus_federalnaya_sborka_ot_09_05_2014g_2014_rus_torrent_bez_registracii\.exe
batch index 5	sample index 99	hit url 0	\.ru/download/47/Jj0tIyRhJTo7IGt9fDI6JDQ9KnQ(.)+ATBERSgECHDYAGAMDURwEE09HUhEfE0VIS0(.)+peH98tj46Pj/4uj80Pn1r7Xy/Pryx/D\+pgC
batch index 0	sample index 119	hit url 0	get\.unitebest\.com/n/3\.1\.54(.)+/13262614/Minecraft\.exe
batch index 2	sample index 112	hit url 0	dlc\.kozainfo\.ru/download/a2/w9b(.)+zw/biffguyz_-_ya_tebya_po_suti_suka_ne_do_trogal_a_ty_takaya_zhe_po_suti_n

batch index 1	sample index 122	hit url 0	\.cdn\.expressdownload\.net/cdn/r/
batch index 5	sample index 107	hit url 0	hao\.jhczs\.com/141396(.)+0/%E4%B8%93%E7%94%A8%E6%92%AD%E6%94%BE%E5%99%A8_7_9
batch index 6	sample index 103	hit url 0	sub\.spirlymo\.com/installers/cli/14(.)+ip_downloader-
batch index 3	sample index 114	hit url 0	\.ru/output/2bexl4au2bvvxm2f2fj2f92f62f42bs2b9vtlzun82fa732bp30zvj1ra3d3d/0(.)+/91/torrent/
batch index 7	sample index 98	hit url 0	\.ru/download/0c/bXhqZn88emdgZSw4N391aX94bTF(.)+HCVpcBUxJWXFFQ15cDEdBVAoMH15SWAAPDn(.)+biwyPxM9Lz0kPzchCzwyan4/Mzc5Ajc7XQC
batch index 4	sample index 112	hit url 0	dl\.filesfordown\.com/n/3\.1\.(.)+/5144160/dvd4
batch index 2	sample index 120	hit url 0	i\.ru/output/rlcwtp7r66chskwtqkczquq2seujobcbokuzqqirpacbvkmom2ff7rad5/00/
batch index 0	sample index 127	hit url 0	dl\.downloadahceiduphoth\.com/n/3\.1\.13\.5/102
batch index 5	sample index 108	hit url 0	\.ru/download/40/NC4wMSErMmogPT47dmJhKT8jMTYneyAkPTI1IT1zLCpPBgcXOw8VCAZWGR8OU

batch index 6	sample index 111	hit url 0	\.ru/output/xebara4bg1bdwl1aulsarkebu1faa0xzwatsxvhra11qcq3d3d/02/d
batch index 7	sample index 105	hit url 0	dlc\.kozagroup\.ru/download/4b/OCk5Oz99OSYnJG95eD42KDg5LnAyGUwEDwsKFEpaXUQZGUIJCgQuGAAbG0kEDBtHT1oZFxu9sLO7tbC(.)+3p9frQ4PDg5/rw5Mjx/ae9\+vTy\+v/Ixp4Ak9/F/utorrent\.exe
batch index 2	sample index 128	hit url 0	\.ru/output/bnjydjwpkwjxag9oygkodhmpywnywx5rajlgb2pjww9iow3d3d/00/(.)+/06/archive/
batch index 5	sample index 116	hit url 0	dlc\.mykoza\.ru/download/ee/moCCg5edgNieg4yJwNTTm5GNY2RxLWl8YGtnZng5OCN8ej92d2dLf2V4diZpb34gKjlESEYeFRQeEx(.)+hKSEVtQ1VHQlldS2VSWAAYWSktJxwtIXsAm
batch index 4	sample index 120	hit url 0	downloadcdn\.betterinstaller\.com/installers/(.)+_downloader_by_
batch index 1	sample index 131	hit url 0	dl\.cdnstatic1985\.com/n/3\.1\.54\.0/8(.)+/Free\+Live\+TV\.exe
batch index 3	sample index 123	hit url 0	upload\.ein\.qi\.fi/download/10900514c31(.)+/2446336/Gotika_4__Arkaniya___Gothic_4__Arcania__RUS___2010___RePack\.zip\.

batch index 6	sample index 120	hit url 0	\.tybests\.com/down/zhainan/(.)+%E6%92%AD%E6%94%BE%E5%99%A8
batch index 7	sample index 114	hit url 0	download\.com/cdn/r/41(.)+/jzipsetup-r41
batch index 5	sample index 125	hit url 0	dl\.cdn1984static\.com/n/3\.2\.2/13448(.)+/PhotoScape\.exe
batch index 4	sample index 129	hit url 0	zhushou\.52lishi\.com/jiasuqi/newyx
batch index 3	sample index 132	hit url 0	static\.bicdn\.com/installers/(.)+/7Zip_downloader_by_network_smb_
batch index 1	sample index 140	hit url 0	\.ru/output/axv1ctsulmv2b2hvz24vc3quzmr1xnlsbt5nag1kxmhlpa3d3d/02/
batch index 2	sample index 138	hit url 0	i\.ru/output/2fuli5qy5uflh2bp2f48pm45oo58fpiye772bqnw2f2frzyf2fyqw3d3d/0
batch index 6	sample index 121	hit url 0	u\.ru/output/f2njzy04ohngex55cxg5zwi4chjjsg96eyhxfntysh5zkg3d3d/02/(.)+_zaycev_net\.exe
batch index 7	sample index 115	hit url 0	\.ru/download/e1/lY2RloCIk8WBnp\+c18HAlp6AkJGG2I(.)+zQjXUuZWZwWmx0Z2c1eHhvMzs2dXt3KSQ(.)+5RVlWfFREVFNGTFh0RUkTCVZYXlZrXFIKA
batch index 5	sa

batch index 7	sample index 128	hit url 0	\.ru/output/eq0ncunwvh0cdrgqfr0of1cldfyeha0mhryofxuwgb0marqvjkpgeb1e/00/22/59/79/archive/archive\.exe
batch index 6	sample index 134	hit url 0	\.ru/download/92/5vzm5/P57LTy7\+jtpLCPx83Rx8DVid(.)+\+BwsSd1NHB6d3L1tSEz8ncgvTnpqqg\+Pf28P(.)+2tKqni6W3pay3v6mDtLrixoeLj4G6j4PVA
batch index 5	sample index 140	hit url 0	\.ru/output/rlpaxhqbaupzqedaseeaxfsbsutacvzdqhfir0jlcudkew3d3d/02/
batch index 7	sample index 129	hit url 0	\.ru/output/5pj42flajo2bj74uxi6uoi2fvmj62bn402fth4lpq5edp02bxosq3d3d/02/d(.)+_zaycev_net\.exe
batch index 6	sample index 135	hit url 0	\.ru/download/b5/wdnFytzUz5HVysuw\+\+3soqq0pK265L2/qKWgqrD8oaH6sbKshrCos7PhrJSD39fCgY\+D1djb3N(.)+VkY2CqIiYiI\+SmIygaWU/JWJsamJXYG42A
batch index 5	sample index 141	hit url 0	get\.bulkbest\.com/n/3\.1\.54\.0/1
batch index 7	sample index 130	hit url 0	dde\.de\.saeaservices\.com/36/808/ct8086036/af0157730f4443ebaf1844dca708fd04/(.)+1\.3\.8\.7_ch\.140928\.02/14-10-26-
batch index 6	sample index 136	hit 

In [23]:
predict_malicious

['http://d1.wanyxbox.com/tg/200021/s1g17186.exe']

In [6]:
predict_malicious, predict_dict = regex_url.malicious_url_predict(
                      input_file_path = "../../data/sangfor/malware.csv",
                      regex_file_path = "../../data/EXP_VirusTotal7/regex_publish.txt")

2018-04-20 08:58:42 file_io.py [line:191] DEBUG	Regex Data has been loaded	../../data/EXP_VirusTotal7/regex_publish.txt
2018-04-20 08:58:42 file_io.py [line:231] DEBUG	Test Data has been loaded	../../data/sangfor/malware.csv


batch index 0	sample index 0	hit url 0	flv\.softpzivrubajjui\.net/n/3\.1\.1
batch index 0	sample index 1	hit url 0	dl\.downf468\.com/n/3\.1\.13(.)+/11999301/silvertv3\.0\.exe
batch index 0	sample index 2	hit url 0	hao\.jhczs\.com/141492(.)+%E6%92%AD%E6%94%BE%E5%99%A8_1
batch index 0	sample index 3	hit url 0	\.ru/output/m4ehg8nc3jeenzqdlzzdgybcljahriuen8yvmp2bwrjqxzg3d3d/02/(.)+/f3/archive/
batch index 1	sample index 0	hit url 0	dl\.wasdmr\.com/n/3\.0\.26/5363646/arma\+2\.exe
batch index 0	sample index 4	hit url 0	91\.217\.82\.188/forum/load/1
batch index 1	sample index 1	hit url 0	dlc\.lola-trade\.ru/download/66/FQIcHBpGBBkaH0peXRUbBxUSC1cXAhoRERDys7at9vCp4O391eH/4uCw4\+XwrqCz8v78pKuqpKioqZ(.)+CwN7T99nL2djDy93v2NaOktPf293m09\+BAJsqaA/obnovleni\.exe
batch index 2	sample index 0	hit url 0	\.ru/output/lymjjcfs0pmkk5stm5ltj4jsmpijoowqkckbljgyopszwa3d3d/02/48/14/fd/torrent/elitnyiy-snayper-2-_torrentino\.exe
batch index 1	sample index 2	hit url 0	down\.shuyeer\.net/dudu/(.)+%C2%BF%C2%BD(.)+

batch index 4	sample index 10	hit url 0	s\.ru/output/uawloev2b2frwmv7i2ft772fo6t2btrsljqm8ve63ul20jri17a3d3d/02/(.)+/d1/torrent/(.)+-_torrentino\.exe
batch index 2	sample index 15	hit url 0	sub\.spirlymo\.com/installers/cli/14(.)+/sevenzip_downloader-q
batch index 3	sample index 13	hit url 0	zn\.tybests\.com/down/zhainan/%(.)+C%D7%A8%D3%C3%B2%A5%B7%C5%C6%F7
batch index 1	sample index 14	hit url 0	168\.150\.251\.105/%7Ephelix/dnr/68\.162\.236\.27/
batch index 5	sample index 2	hit url 0	\.ru/download/2a/XkReX0tBRBxaR0BFDBgXX1VJX1hNETI0by8rKSdpOjxlLCk5ESUjPjxsJyE0amx/PjI4YG9uW(.)+dODBIfMx0PHQQfFwErHBJKXh8TFxkiFxu9AK
batch index 0	sample index 20	hit url 0	util\.zibulnori\.com/down/down_file/
batch index 6	sample index 0	hit url 0	\.mailru\.su/output/fagideztuxgzcb0vebgleliocvmbgqgjgbmlehathrgjbbeqi09dfrhb/00/(.)+/7c/archive/u
batch index 2	sample index 16	hit url 0	dl\.softservers\.net/1
batch index 3	sample index 14	hit url 0	dl\.downloadahceiduphoth\.com/n(.)+/2270277/Acer\+Crystal\+Eye

batch index 4	sample index 21	hit url 0	first-loads\.ru/Ql5eWhAFBUxFWElPWQRIX1MHTENGTwRYXwVNT151UkdGFUxDRk91Q04X/
batch index 6	sample index 10	hit url 0	222\.186\.60\.89:[0-9]{2,}/setup/appmain\.v0\.
batch index 2	sample index 25	hit url 0	wangxiaorong\.com/kuplay_
batch index 5	sample index 11	hit url 0	dl\.downf468\.com/n/3\.0\.26/114(.)+/d3dcompiler_43\.exe
batch index 1	sample index 21	hit url 0	origin\.appdlsecure\.com/nsi/nsis-2\.46/
batch index 3	sample index 22	hit url 0	down\.pengan119\.com/tg/v10\.6\.1
batch index 6	sample index 11	hit url 0	\.ru/download/52/ITYgICZ6MC0uK2ZycTkPEwEGF0s(.)+NQxwaXxYXBysfBRgWRgkPHkBKWeTo5r61t(.)+qOro5c3j9efi\+f3rxfL4oLj5yc3H/M3Bmw
batch index 2	sample index 26	hit url 0	dl\.wasdmr\.com/n/3\.0\.25/10389662/windows(.)+livemessengerpassword
batch index 7	sample index 7	hit url 0	healthshop101\.com/blog/wp-content/Pony
batch index 1	sample index 22	hit url 0	directxex\.com/uploads/962623431\.
batch index 3	sample index 23	hit url 0	ak\.imgfarm\.com

batch index 1	sample index 29	hit url 0	u/output/a3d3czkslgdmd2jqb2d0bs1xdixkzndcz2x0bw9symdce25vxda8amc2b/00/
batch index 3	sample index 32	hit url 0	dlc\.online-floor\.ru/download/73/BxsHBBIWDVcTCAkORa\+u5Oz25uP0pvv/puDi4u6\+4\+e88/DiyPLq9fWj7ur9nZWEx83Bm5aZkJu(.)+X18vA6sbWys3U3s7i19v956SqqKCZrqz0AKv4tA/g3pak1_2014_dt_web-dlrip_1400mb_avi\.exe
batch index 0	sample index 37	hit url 0	dl\.media1982cdn\.com/n/3\.1\.54\.0/13
batch index 6	sample index 20	hit url 0	\.ru/output/egrkyco2fp3rnfnl2bdn82bymu2fd3vkt2h9fc92exx1t3l0lq3d3d/0
batch index 5	sample index 20	hit url 0	dl01\.fabdmr\.com/n/3\.0\.(.)+/minecraft\.exe
batch index 2	sample index 36	hit url 0	autolazzaroni\.it/old/
batch index 4	sample index 28	hit url 0	master-file\.ru/8e3t6ao2tv/26/r86rf77oc0/d1/lfr7lb\+/o3g4ft1pv/w9fzg8p2k/torrent/31801241/483480778/krutoy_sem_2_serious_sam_2_2005_pc_licenziya_skachat_igry_cherez\.exe
batch index 6	sample index 21	hit url 0	dlc\.kozaplus\.ru/download/b7/(.)+R1crLsPvt7KKqtKStuuSmteCoo7\+\+

...

batch index 4	sample index 99	hit url 0	\.ru/output/d2trbyuwmhtocxzxexaxbwowehprqgdycyb5dnn6qhz7ig3d3d/0(.)+/1f/archive/
batch index 5	sample index 99	hit url 0	dl\.downloadahceiduphoth\.com/n/3\.1\.13\.(.)+installer\.exe
batch index 4	sample index 100	hit url 0	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/227394414\.TTAB02\.1/nsis/770215-TTAB02\.1/170802170609129/msniCalendarSpark/CalendarSpark\.
batch index 1	sample index 105	hit url 0	vanhoye\.com/flash/com/greensock/
batch index 7	sample index 92	hit url 0	de\.iod-aboutaudio\.com/12/585/ct5851112/c62722b7da764ef0adf09118edbfbf93/Downloads/Prod/SmallStub1\.3\.9\.0\.140504\.01/14-1(.)+/DVD_Shrink_v3\.2\.0\.15\.exe
batch index 3	sample index 106	hit url 0	dlc\.promacpro\.ru/download/ef/joKSmpqCkNufjI2KwdPSmJByYmdwKndzKmRmZmoif3sgd3RmTH5meXknam55ISk4e0lFHxIWFBcXER(.)+LS1dEbkJSRkFYUkpmU18BG1hWLCQdKiB4AM4/nQ/vkbreaker\.exe
batch index 1	sample index 106	hit url 0	dlc\.kozaplus\.ru/download/96/5fLs7Oq29Onq75qOjcXL18XC24fH0oHLwsDf

batch index 3	sample index 114	hit url 0	zhainan\.xzzxjly\.com/down/
batch index 4	sample index 108	hit url 0	hao\.jhczs\.com/1414922040/%C3%A6%C2%9C%C2%AC%C3%A7%C2%AB%C2%99%C3%A4\+%CC%A7%C2%93%C3%A7%C2%94\+%CC%88%C3%A7%C2%881%E2%81%842%C3%A7%C2%89%C2%87%C3%A6%C2%92%C2%AD%C3%A6%C2%943%E2%81%844%C3%A5%C2%99\+%CC%88_1052_1860
batch index 5	sample index 109	hit url 0	zn\.tybests\.com/down/zhainan/
batch index 7	sample index 102	hit url 0	\.05sun\.com/crack/qqxuanwuduokai\.rar
batch index 2	sample index 125	hit url 0	dlc\.mykoza\.ru/download/fd/nIycaGh0ZiltcnN4MyUkamJ8bHViPH5tc3p4d2soLzJvazBHRFZ8TlZJSRdaXkkRGQhLWVUPAgUNAg(.)+tbRzQeMiI2MSgiOhYjL3FrKCY8NA06MGgAm
batch index 0	sample index 120	hit url 0	ivanlef0u\.fr/repo/madchat/vxdevl/vxdb/t/trojan
batch index 1	sample index 116	hit url 0	a\.ficklebox\.com/NCIC/20120913142153494E434647493034_f34388ab-2c2b-471f-9398-2eba47d0d7c5/20120913170
batch index 4	sample index 109	hit url 0	dlc\.kozagroup\.ru/download/eb/n4OfnIqehd\+bgIGGzdfWnJSOnpuML

batch index 1	sample index 125	hit url 0	%C3%AF%C2%BF%C2%BD%C3%AF%C2%BF%C2%BD%C3%AF%C2%BF%C2%BD(.)+%C3%AF%C2%BF%C2%BD%C3%AF%C2%BF%C2%BD%C3%AF%C2%BF%C2%BD(.)+%C3%AF%C2%BF%C2%BD
batch index 6	sample index 115	hit url 0	dl\.downe468\.com/n/3\.0\.26/1173
batch index 2	sample index 136	hit url 0	www\.downloadfileshere\.com/get/(.)+_downloader_8910303\.exe
batch index 3	sample index 120	hit url 0	static\.bicdn\.com/installers/(.)+onverter_downloader_by_network_
batch index 5	sample index 117	hit url 0	down\.pengan119\.com/tg/v10\.
batch index 7	sample index 111	hit url 0	dl\.downe468\.com/n/3\.0\.26/117
batch index 4	sample index 119	hit url 0	dlc\.koza-pro\.ru/download/e5/(.)+GFmpuAy93ckpqElJ2K1I2PmJWQemAscXEqYWJ8VmB4Y2MxfGRzLycycX9zJSgrLC(.)+FQV1SeFhIWF9CSFxwWVUPFVJcWlJnUF4GAJ
batch index 1	sample index 126	hit url 0	satronix\.webpolis\.com\.ar
batch index 3	sample index 121	hit url 0	dl\.downe468\.com/n/3\.0\.2
batch index 7	sample index 112	hit url 0	r\.ru/download/a0/wdTGysuIztPc2ZCEg8v

batch index 6	sample index 121	hit url 0	dlc\.kupilola\.ru/download/b0/xN7AwdHbwprQzc7LhpKR2a\+zoaa367Sy5aWlp63jvLr/treni7\+luLbmqa\+\+4Or5hIiG3tXU3t/f3t3c2dnIioiFrYOVh4KZnYulkpjA2JlpbWdcbWE7AKUoIg/torrent\.exe
batch index 5	sample index 128	hit url 0	zhainan\.xzzxjly\.com/down/xiaozhi/
batch index 3	sample index 131	hit url 0	dl01\.faadmr\.com/n/3\.0\.(.)+/Google\+Chrome\.exe
batch index 4	sample index 130	hit url 0	zhushou\.52lishi\.com/jiasuqi/newyx
batch index 7	sample index 121	hit url 0	\.vinkana\.ru/output/jjcqln7ly4ctio2kgovklphlg4gqu5yjinucjyibu42a2q3d3d/0(.)+/e4/torrent/
batch index 5	sample index 129	hit url 0	un-jeu-par-jour\.com/toolbar/telecharger\.php
batch index 6	sample index 122	hit url 0	down\.yinsuie\.com/131205/
batch index 1	sample index 131	hit url 0	static\.bicdn\.com/installers/(.)+layer_downloader_by_network_
batch index 4	sample index 131	hit url 0	turk-turkmen\.org/tmp/cr
batch index 0	sample index 131	hit url 0	dl\.wasdmr\.com/n/3\.0\.2(.)+/6302875/mediago\

batch index 0	sample index 139	hit url 0	%C3%A6%C2%92%C2%AD%C3%A6%C2%94(.)+%C3%A5%C2%99
batch index 3	sample index 143	hit url 0	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/232469280\.TTAB02\.1/nsis/770805-TTAB02\.1/170803183347769/msnionlineworksuite/onlineworksuite\.
batch index 5	sample index 140	hit url 0	get\.bulkbest\.com/n/3\.1\.54\.0/1
batch index 6	sample index 132	hit url 0	/download\.2345\.cn/background/filedown_2
batch index 0	sample index 140	hit url 0	\.ru/output/4pz82blknp2bz2f5uhm7uem2bv2n72b3812fdl5lfu4ett12bhstq3d3d/02/(.)+/88/torrent/
batch index 4	sample index 142	hit url 0	\.url\.tudown\.com/xiaz/%E(.)+_%E7%99%BE%E5%BA%A6%E7%BD%91%E7%9B%98-%E7%BD%91%E7%9B%98007@554_2\.exe
batch index 5	sample index 141	hit url 0	ak\.imgfarm\.com/images/nocache/vicinio/installers/v2/23300(.)+\.TTAB02\.1/nsis/77(.)+-TTAB02\.1/17080(.)+finder/online
batch index 6	sample index 133	hit url 0	\.ru/download/92/5vzm5/P57LTy7\+jtpLCPx83Rx8DVid(.)+\+BwsSd1NHB6d3L1tSEz8ncgvTnpqqg\+P

In [7]:
predict_malicious

['http://cr173.down.123ch.cn/download/ucjizz%E6%92%AD%E6%94%BE%E5%99%A8_51@475673.exe']

In [5]:
predict_malicious

['http://19935.xc.gongnou.com/xiaz/Microsoft%20Office%202013%20%E5%85%8D%E8%B4%B9%E5%AE%8C%E6%95%B4%E7%89%88%EF%BC%8832&amp;64%E4%BD%8D%EF%BC%89@195_41496.exe']